In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt # Plotting

from gvxrPython3 import gvxr # Simulate X-ray images`
from gvxrPython3 import json2gvxr # Set the simulation parameters using a JSON file
from utilities import recon_parallel, recon_widget, recon_cone

spekpy is not install, you won't be able to load a beam spectrum using spekpy
SimpleGVXR 2.0.2 (2022-10-07T18:07:14) [Compiler: GNU g++] on Linux
gVirtualXRay core library (gvxr) 2.0.2 (2022-10-07T18:07:13) [Compiler: GNU g++] on Linux


In [2]:
json2gvxr.initGVXR("Turbopump.json", "OPENGL")

Create an OpenGL context: 450x450


Fri Oct 14 21:27:24 2022 ---- Create window (ID: -1)
Fri Oct 14 21:27:24 2022 ---- Initialise GLFW
Fri Oct 14 21:27:24 2022 ---- Create an OpenGL window with a 3.2 context.
Fri Oct 14 21:27:25 2022 ---- Make the window's context current
Fri Oct 14 21:27:25 2022 ---- Initialise GLEW
Fri Oct 14 21:27:25 2022 ---- OpenGL vendor: NVIDIA Corporation
Fri Oct 14 21:27:25 2022 ---- OpenGL renderer: NVIDIA GeForce RTX 2080 Ti/PCIe/SSE2
Fri Oct 14 21:27:25 2022 ---- OpenGL version: 3.2.0 NVIDIA 510.73.08
Fri Oct 14 21:27:25 2022 ---- Use OpenGL 4.5.
Fri Oct 14 21:27:25 2022 ---- Initialise the X-ray renderer if needed and if possible


In [3]:
json2gvxr.initDetector()

Set up the detector
	Detector position: [0, -150, 0, 'mm']
	Detector up vector: [0, 0, 1]
	Detector number of pixels: [900, 900]
	Pixel spacing: [0.2, 0.2, 'mm']


In [5]:
json2gvxr.initSamples()

Fri Oct 14 21:28:03 2022 ---- file_name:	../input_data/TurboPump/internals.stl	nb_faces:	86276	nb_vertices:	258828	bounding_box (in cm):	(-3.48613, -5.55, -3.48613)	(3.48613, -0.735, 3.48613)
Fri Oct 14 21:28:03 2022 ---- file_name:	../input_data/TurboPump/front_flange.stl	nb_faces:	6120	nb_vertices:	18360	bounding_box (in cm):	(-11.425, 0, -11.425)	(11.425, 3.535, 11.425)
Fri Oct 14 21:28:03 2022 ---- file_name:	../input_data/TurboPump/rear_flange.stl	nb_faces:	10516	nb_vertices:	31548	bounding_box (in cm):	(-11.425, -15.535, -11.425)	(11.425, -6.73, 11.425)
Fri Oct 14 21:28:03 2022 ---- file_name:	../input_data/TurboPump/housing.stl	nb_faces:	35076	nb_vertices:	105228	bounding_box (in cm):	(-12.3086, -8.5025, -14.9408)	(16.5171, 1.8975, 14.8965)
Fri Oct 14 21:28:03 2022 ---- file_name:	../input_data/TurboPump/ThrustRollerBearing.stl	nb_faces:	26384	nb_vertices:	79152	bounding_box (in cm):	(-6.25, -9.27, -6.25)	(6.25, -6.77, 6.25)


In [6]:
json2gvxr.initSourceGeometry()
json2gvxr.initSpectrum();

Set up the beam
	Source position: [0, 200, 0, 'mm']
	Source shape: Point


Fri Oct 14 21:28:09 2022 ---- Initialise the renderer


In [7]:
# gvxr.computeXRayImage()
# gvxr.displayScene()
# gvxr.renderLoop()

# screenshot = gvxr.takeScreenshot()
# plt.imsave("../img/turbo_pump-screenshot.png", np.array(screenshot))

In [ ]:
xray_image_point_source = gvxr.computeXRayImage()
gvxr.displayScene()
screenshot_point_source = gvxr.takeScreenshot()

In [ ]:
theta = [];
num_projections = 721
angular_step = 360 / num_projections

# Compute the intial X-ray image (zeroth angle) and add it to the list of projections
xray_image = np.array(gvxr.computeXRayImage()).astype(np.single)

# Flat-field correction
xray_image /= gvxr.getTotalEnergyWithDetectorResponse()

imwrite('output_data/pump_scan/03-Pump_0.tiff',xray_image)

projections = []

# Update the 3D visualisation
gvxr.displayScene();
theta.append(0.0);
for i in range(1, num_projections):

    #    Rotate the model by angular_step degrees
    gvxr.rotateNode("root", -1 * angular_step, 0,0,1);

    # Compute an X-ray image and add it to the list of projections
    xray_image = np.array(gvxr.computeXRayImage()).astype(np.single)
    
    # Flat-field correction
    xray_image /= gvxr.getTotalEnergyWithDetectorResponse()
    projections.append(xray_image)

    # Update the 3D visualisation
    gvxr.displayScene();

    imwrite(f'output_data/pump_scan/03-Pump_{i}.tiff',xray_image)
    theta.append(i * angular_step * math.pi / 180);

In [ ]:
file = open("GVXR_angles.txt", "w")
file.write(f"angles = {theta} \n")
file.close()

In [ ]:
gvxr.terminate()